In [2]:
from bs4 import BeautifulSoup
from cltk.stem.lemma import LemmaReplacer
from cltk.tokenize.word import WordTokenizer
from collections import defaultdict
import json
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
import os
import re
import string
from cltk.tokenize.sentence import TokenizeSentence
from nltk.tokenize import sent_tokenize
import bs4


In [30]:
def make_list(soup):
    l = []
    for n in soup:
        try:
            l.append(int(n['n']))
        except:
            l.append(-1)  
    return remove_duplicates(l)


def remove_duplicates(l):
    i = 0
    while i+1 < len(l):
        if l[i] == l[i+1]:
            l[i+1] += 1
        if l[i] > l[i+1] and l[i+1] != -1:
            l[i] = l[i+1]-1
        i += 1
    return l

def match(large,small):
    l = make_list(large)
    s = make_list(small)
    li = 0
    si = 0
    groups = []
    #print(l,s)
    while si < len(s) and li < len(l):
        if s[si] == -1:
            si += 1
            continue
        if l[li] == -1:
            li += 1
            continue
        lgroup = []
        sgroup = []
        if l[li] == s[si]:
            lgroup.append(li)
            sgroup.append(si)
              
        if si+1 < len(s) and li+1 < len(l): 
            if s[si+1] == -1:
                si += 1
            
            if l[li+1] == -1:
                li += 1
            
            
            if l[li+1] != s[si+1]:
                while l[li+1] != s[si+1]:
                    if si+1 < len(s) and li+1 < len(l):
                        if l[li+1] > s[si+1]:
                            sgroup.append(si+1)
                            si += 1
                            if s[si] == -1:
                                if si+1 < len(s):
                                    si += 1
                                else:
                                    pass
                        else:
                            lgroup.append(li+1)
                            li += 1
                            if l[li] == -1:
                                if li+1 < len(l):
                                    li += 1
                                else:
                                    pass
                    else:
                        break
                si += 1
                li += 1
                groups.append([sgroup,lgroup])
            else:
                si += 1
                li += 1
                groups.append([sgroup,lgroup]) 
        else:
            break        
    
    groups.append([sgroup,lgroup])
    return groups
        

def align_soups(m,small,large):
    s_alignments = []
    l_alignments = []
    for g in m:
        s = []
        l = []
        for i in g[0]:
            s.append(small[i])
        for i in g[1]:
            l.append(large[i])
        s_alignments.append(s)
        l_alignments.append(l)   
    return s_alignments,l_alignments


def get_pairs(latin_soup, eng_soup, tag, typename, typos):
    """ Given two soups, returns soups of equal size, with each element corresponding to the same
        element in the other soup.
        Lists may contain further lists of soups which are aligned with one another, they are combined into one list
        in the first step of this function.
    """
    lat_sect,eng_sect = [], []
    
    for i in range(len(latin_soup)):
        lat_sect += latin_soup[i].findAll(tag, {typename: typos})
    for i in range(len(eng_soup)):
        eng_sect += eng_soup[i].findAll(tag, {typename: typos})

    if len(lat_sect) > len(eng_sect):
        try:
            #print(latin_soup[0]['n'],latin_soup[0].parent['n'])
            m = match(lat_sect,eng_sect)
            eng_alignments,lat_alignments = align_soups(m,eng_sect,lat_sect)
            return lat_alignments,eng_alignments
        except:
            print("error with {}, {}".format(tag,typos))
    elif len(lat_sect) < len(eng_sect):
        try:
            #print(latin_soup[0]['n'],latin_soup[0].name)
            m = match(eng_sect,lat_sect)
            lat_alignments,eng_alignments = align_soups(m,lat_sect,eng_sect)
            return lat_alignments,eng_alignments
        except:
            print("error with {}, {}".format(tag,typos))
    else:
        if eng_sect[0]['n'] == 'intro':
            try:
                #print(latin_soup[0]['n'],latin_soup[0].name)
                m = match(eng_sect,lat_sect)
                lat_alignments,eng_alignments = align_soups(m,lat_sect,eng_sect)
                return lat_alignments,eng_alignments
            except:
                print("error with {}, {}".format(tag,typos))
        else:
            
            return lat_sect, eng_sect
    



def align_pars(lsentences,esentences):
    
    if lsentences[-1] == '':
        lsentences = lsentences[:-1]
    if esentences[-1] == '':
        esentences = esentences[:-1]
        
    if len(lsentences) > len(esentences):
        alignments = align(lsentences,esentences)
        target = esentences
        source = lsentences
    elif len(lsentences) < len(esentences):
        alignments = align(esentences,lsentences)
        target = lsentences
        source = esentences
    else:
        return zip(lsentences,esentences)
    
    sentence_tuples = []
    for k in alignments.keys():
        sentence_tuples.append((target[k], [source[j] for j in alignments[k]]))
        
    return sentence_tuples
          

def get_text(soup):
    """Expects a soup in the form of a find('p') selection"""
    s = []
    for c in soup.children:
        if c.name in [None,'corr', 'name']:
            try:
                s.append(c.text.strip())
            except:
                s.append(c.strip())
    return s


def clean_lemma(word):
    try:
        word = re.sub('[0-9]+', '', word)
        word = re.sub('[-_]+', '', word)
    except:
        pass

    return word


def get_lemmata_defs(latin):
    words = lemmatizer.lemmatize(latin)
    ldefs = []
    unmatched = []

    for word in words:
        word = clean_lemma(word)
        try:
            ldefs.append(defs[word]['definition'].split(','))
        except:
            unmatched.append(word)
    return ldefs,unmatched


def get_defs(ldefs):
    lldefs=[]
    for l in ldefs:
        lldefs += t(l[0])
    return lldefs


def get_match_count(english,latin):
    
    ldefs,unmatched = get_lemmata_defs(latin)
    ldefs = get_defs(ldefs)
    count = 0
    for word in t(english):
        if word in string.punctuation:
            continue
        if word in ldefs and word not in stop:
            count += 1
        if word in unmatched:
            count += 1
    return count


def align(source,target):
    """ Takes a paragraph of sentences, source, and maps each sentence to a sentence in the 
        target paragraph. 
        
        Returns a dictionary with the target sentence indices as keys, and the sentences in the source paragraph
        which correspond to them as values.
    """
    tsize = len(target)
    ssize = len(source)
    alignments = defaultdict(list)
    alignments[0] = [0]
    i = 1
    j = 1
    while tsize < ssize and i < len(source) and j < tsize:
        max_count = get_match_count(source[i],target[j])
        max_i = j
        if j >= tsize-1:
            rge = [j-1]
        else:
            rge = [j-1,j+1]
        for r in rge: 
            c = get_match_count(source[i],target[r])
            if c > max_count:
                max_count = c
                max_i = r

        if max_i != j:
            ssize -= 1
        else:
            j += 1

        alignments[max_i].append(i)
        i += 1
        
  
    while j < tsize:
        alignments[j].append(i)
        j += 1
        i += 1
    alignments[j-1].extend([k for k in range(i,len(source))])
    
    return alignments

def get_sec_alignments(lpar,epar):
    if len(lpar) > len(epar):
        if len(epar) > 1:
            alignments = align(lpar,epar)
            aligned = []
            for k in alignments.keys():
                group = []
                for v in alignments[k]:
                    group.append(lpar[v])
                aligned.append((group,epar[k]))
        else:
            try:
                aligned = [('. '.join(lpar),epar[0])] 
            except:
                print('error, {}'.format(epar))
                aligned = list(zip(lpar,epar))
    elif len(epar) > len(lpar):
        if len(lpar) > 1:
            alignments = align(epar,lpar)
            aligned = []
            for k in alignments.keys():
                group = []
                for v in alignments[k]:
                    group.append(epar[v])
                aligned.append((lpar[k],group))
        else:
            try:
                aligned = [(lpar[0],'. '.join(epar))]
            except:
                print('error, {}'.format(lpar))
                aligned = list(zip(lpar,epar))
    else:
        aligned = list(zip(lpar,epar))
        
    return aligned

def get_sentences(chap_soup, sect_soup):
    t = bs4.element.Tag
    sect = []
    sentences = []
    i = 0
    c = 0
    children = list(chap_soup.children)
    soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
    c += 1
    while type(soup) != t:
        soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
        c += 1
    i += 1
    while c < len(children):
        while soup.next_sibling != None:
            soup = soup.next_sibling
            if soup.name == 'milestone':
                try:
                    if int(soup['n']) == int(sect_soup[i]['n']):
                        i += 1
                        sentences.append(' '.join(sect))
                        sect = []
                except:
                    soup = soup.next_sibling
                    i += 1
            if soup.name in ['corr', 'name','hi','quote','reg', 'l', 'lg', 'placeName']:
                    sect.append(soup.text.strip())
            elif soup.name == None:
                    sect.append(soup.strip())
                    
        sentences.append(' '.join(sect))
        sect = []
        try:
            soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
            c += 1
            while type(soup) != t and c < len(children):
                soup = children[c].find('milestone')
                c += 1
            i += 1
        except:
            continue
    sentences.append(' '.join(sect))
    sect = []
    return sentences

### Aulus Gellius ###
	gel_eng.xml , gel_lat.xml  
### Quintilian  
	quint.butler1-3_eng.xml , quint.butler1-3_lat.xml  
	quint.butler10-12_eng.xml , quint.butler10-12_lat.xml  
	quint.butler4-6_eng.xml , quint.butler4-6_lat.xml  
	quint.butler7-9_eng.xml , quint.butler7-9_lat.xml  
### Celsus  
	cels_eng.xml , cels_lat.xml  
### Ammianus Marcellinus  
	amm_eng.xml , amm_lat.xml  
### M. Annaeus Lucanus  
	pharsalia_eng.xml , pharsalia_lat.xml  
### Q. Horatius Flaccus (Horace)  
	hor.ap_eng.xml , hor.ap_lat.xml  
	hor.sat_eng.xml , hor.sat_lat.xml  
### M. Tullius Cicero  
	cic.amic_falc_eng.xml , cic.amic_falc_lat.xml  
	cic.div_falc_eng.xml , cic.div_falc_lat.xml  
	cic.off_eng.xml , cic.off_lat.xml  
	cic.sen_falc_eng.xml , cic.sen_falc_lat.xml  
### Seneca  
	sen.apoc_eng.xml , sen.apoc_lat.xml  
### P. Vergilius Maro  
	verg.ecl_eng.xml , verg.ecl_lat.xml  
	verg.g_eng.xml , verg.g_lat.xml  
### Vitruvius Pollio  
	vitruv_eng.xml , vitruv_lat.xml  
### Sallust  
	sallust.catil_eng.xml , sallust.catil_lat.xml  
	sallust.jugur_eng.xml , sallust.jugur_lat.xml  
### T. Maccius Plautus  
	pl.am_eng.xml , pl.am_lat.xml  
	pl.as_eng.xml , pl.as_lat.xml  
	pl.aul_eng.xml , pl.aul_lat.xml  
	pl.bac_eng.xml , pl.bac_lat.xml  
	pl.capt_eng.xml , pl.capt_lat.xml  
	pl.cas_eng.xml , pl.cas_lat.xml  
	pl.cist_eng.xml , pl.cist_lat.xml  
	pl.cur_eng.xml , pl.cur_lat.xml  
	pl.epid_eng.xml , pl.epid_lat.xml  
	pl.men_eng.xml , pl.men_lat.xml  
	pl.mer_eng.xml , pl.mer_lat.xml  
	pl.mil_eng.xml , pl.mil_lat.xml  
	pl.mos_eng.xml , pl.mos_lat.xml  
	pl.per_eng.xml , pl.per_lat.xml  
	pl.poen_eng.xml , pl.poen_lat.xml  
	pl.ps_eng.xml , pl.ps_lat.xml  
	pl.rud_eng.xml , pl.rud_lat.xml  
	pl.st_eng.xml , pl.st_lat.xml  
	pl.trin_eng.xml , pl.trin_lat.xml  
	pl.truc_eng.xml , pl.truc_lat.xml  
### Lucretius  
	lucretius_eng.xml , lucretius_lat.xml  
### P. Terentius Afer (Terence)  
	ad_eng.xml , ad_lat.xml  
	an_eng.xml , an_lat.xml  
	eu_eng.xml , eu_lat.xml  
	hau_eng.xml , hau_lat.xml  
	hec_eng.xml , hec_lat.xml  
	ph_eng.xml , ph_lat.xml  
### Petronius  
	petr_eng.xml , petr_lat.xml  
### C. Julius Caesar  
	caes.bc_eng.xml , caes.bc_lat.xml  
	caes.bg_eng.xml , caes.bg_lat.xml  
### P. Ovidius Naso  
	ovid.am_eng.xml , ovid.am_lat.xml  
	ovid.met_eng.xml , ovid.met_lat.xml  

In [3]:
t = wordpunct_tokenize
stop = set(stopwords.words('english'))

f = open('lemmas.json')
defs = json.loads(f.read())

lemmatizer = LemmaReplacer('latin')
tokenizer = WordTokenizer('latin')
s_tokenizer = TokenizeSentence('latin')

f = open('../Latin/Ammianus/opensource/amm_lat.xml')
latin_soup = BeautifulSoup(f.read(),'lxml')
f.close()

f = open('../Latin/Ammianus/opensource/amm_eng.xml')
eng_soup = BeautifulSoup(f.read(),'lxml')
f.close()

In [22]:
units = []
for s in latin_soup.find('encodingdesc').findAll('state'):
    units.append(s['unit'])

In [23]:
units

['book', 'chapter', 'section', 'page']

In [4]:
lat,eng = get_pairs([latin_soup],[eng_soup],'div1','type','book')

In [164]:
#lat,eng = get_pairs([latin_soup],[eng_soup],'div1','type','chapter')

In [8]:
lchaps = []
echaps = []
for i in range(len(lat)):
    try:
        p = get_pairs([lat[i]],[eng[i]],'div2','type','chapter')
        if type(p[0][0]) == list:
            lchaps.append([s[0] for s in p[0]])
            echaps.append([s[0] for s in p[1]])
        else:
            lchaps.append(p[0])
            echaps.append(p[1])
    except:
        print('error with {}'.format(i))

In [811]:
lsoup = []
esoup = []
for b in range(len(lchaps)):
    lchap_soup = []
    i = 0
    soup = re.split(r'{}'.format(lchaps[b][i]),str(lat[b]))[1]
    i += 1

    while i < len(lchaps[b]):
        soups = re.split(r'{}'.format(lchaps[b][i]),soup)
        lchap_soup.append(BeautifulSoup(soups[0],'lxml'))
        soup = soups[1]
        i += 1

    lchap_soup.append(BeautifulSoup(soup,'lxml'))

    echap_soup = []
    i = 0
    soup = re.split(r'{}'.format(echaps[b][i]),str(eng[b]))[1]
    i += 1

    while i < len(lchaps[b]):
        soups = re.split(r'{}'.format(echaps[b][i]),soup)
        echap_soup.append(BeautifulSoup(soups[0],'lxml'))
        soup = soups[1]
        i += 1

    echap_soup.append(BeautifulSoup(soup,'lxml'))
    
    lsoup.append(lchap_soup)
    esoup.append(echap_soup)

In [10]:
lsects = []
esects = []
for i in range(len(lsoup)):
    lsects.append([])
    esects.append([])
    for c in range(len(lsoup[i])):
        try:
            p = get_pairs([lsoup[i][c]],[lsoup[i][c]],'milestone','unit','section')
            lsects[i].append(p[0])
            esects[i].append(p[1])
        except:
            print('error with {}'.format(i))

NameError: name 'lsoup' is not defined

In [11]:
lsects = []
esects = []
for i in range(len(lat)):
    lsects.append([])
    esects.append([])
    for c in range(len(lchaps[i])):
        try:
            p = get_pairs([lat[i]],[eng[i]],'milestone','unit','section')
            lsects[i].append(p[0])
            esects[i].append(p[1])
        except:
            print('error with {}'.format(i))

error with milestone, section
error with 1
error with milestone, section
error with 1
error with milestone, section
error with 1
error with milestone, section
error with 1
error with milestone, section
error with 1
error with milestone, section
error with 1
error with milestone, section
error with 1
error with milestone, section
error with 1
error with milestone, section
error with 1
error with milestone, section
error with 1
error with milestone, section
error with 1
error with milestone, section
error with 1
error with milestone, section
error with 1


In [710]:

def get_sentences_2(chap_soup,sect_soup):
    typ = bs4.element.Tag
    sect = []
    sentences = []
    i = 0
    children = list(chap_soup.children)
    soup = children[0].find('milestone')#, {'n':sect_soup[i]['n']})
    
    while True:
        while soup.next_sibling != None:
            soup = soup.next_sibling
            if soup.name == 'milestone':
                try:
                    if int(soup['n']) == int(sect_soup[i]['n']):
                        i += 1
                        sentences.append(' '.join(sect))
                        sect = []
                except:
                    soup = soup.next_sibling
                    #i += 1
            if soup.name in ['corr', 'name','hi','quote','reg', 'l']:
                    sect.append(soup.text.strip())
            elif soup.name == None:
                    sect.append(soup.strip())
            if type(soup) == typ:       
                if list(soup.children) != None and list(soup.children) != []:
                    try:
                        if 'milestone' in [c.name for c in list(soup.children)]:
                            soup = soup.find('milestone')
                            try:
                                if int(soup['n']) == int(sect_soup[i]['n']):
                                    i += 1
                                    sentences.append(' '.join(sect))
                                    sect = []
                            except:
                                soup = soup.next_sibling
                    except:
                        sect += soup
        soup = soup.find_next()
        if soup == None:
            break
            
        if soup.name == 'milestone':
            try:
                if int(soup['n']) == int(sect_soup[i]['n']):
                    i += 1
                    sentences.append(' '.join(sect))
                    sect = []
            except:
                soup = soup.next_sibling
                #i += 1
        if soup.name in ['corr', 'name','hi','quote','reg', 'l']:
            sect.append(soup.text.strip())
        elif soup.name == None:
            sect.append(soup.strip())
            
    sentences.append(' '.join(sect))
    sect = []

    #sentences.append(' '.join(sect))
    #sect = []
    return sentences


In [767]:
lchap_1 = []
echap_1 = []
#for b in range(len(lchaps)):
#lchap_soup = []
i = 0
soup = re.split(r'{}'.format(lsects[0][0][i]),str(lat[0]),maxsplit=1)[1]
i += 1

while i < len(lsects[0][0]):
    soups = re.split(r'{}'.format(lsects[0][0][i]),soup,maxsplit=1)
    lchap_1.append(BeautifulSoup(soups[0],'lxml'))
    try:
        soup = soups[1]
    except:
        soup = soups[0]
    i += 1

lchap_1.append(BeautifulSoup(soup,'lxml'))

In [773]:
#echap_soup = []
i = 0
soup = re.split(r'{}'.format(esects[0][0][i]),str(eng[0]),maxsplit=1)[1]
i += 1

while i < len(esects[0][0]):
    soups = re.split(r'{}'.format(esects[0][0][i]),soup,maxsplit=1)
    echap_1.append(BeautifulSoup(soups[0],'lxml'))
    try:
        soup = soups[1]
    except:
        soup = soups[0]
    i += 1

echap_1.append(BeautifulSoup(soup,'lxml'))

#lsoup.append(lchap_soup)
#esoup.append(echap_soup)

In [28]:
alignments = []
for b in range(len(lsoup)):
    print('b: ',b)
    sec_alignments = []
    for c in range(len(lsects[b])):
        lch = lsoup[b][c]
        lsec = lsects[b][c][0]
        ech = esoup[b][c]
        esec = esects[b][c][0]
        try:
            ls = get_sentences_2(lch,lsec)
            es = get_sentences_2(ech,esec)
        except:
            print('error, {},{}'.format(b,c))
        ls = [re.sub(r'[0-9]{1,2}\.','',l) for l in ls]
        es = [re.sub(r'[0-9]{1,2}\.','',e) for e in es]
        ls = [re.sub(r'\. \. \.','',l) for l in ls]
        es = [re.sub(r'\. \. \.','',e) for e in es]
        ls =  [re.sub('[:;,]','',l) for l in ls]
        es = [ re.sub('[:;,]','',e) for e in es]

        epar = [sent_tokenize(e.replace('\n',' ')) for e in es]
        lpar = [s_tokenizer.tokenize_sentences(l.replace('\n',' ')) for l in ls]
        if len(lpar) != len(epar):
            print(len(lpar),len(epar))
            continue
        
        for p in range(len(lpar)):
            sec_alignments.append(get_sec_alignments(lpar[p],epar[p]))
    alignments.append(sec_alignments)

NameError: name 'lsoup' is not defined

In [27]:
lsects = []
esects = []
for i in range(len(lchaps)):
    lsects.append([])
    esects.append([])
    for c in range(len(lchaps[i])):
        try:
            p = get_pairs([lchaps[i][c]],[echaps[i][c]],'milestone','unit','section')
            lsects[i].append(p[0])
            esects[i].append(p[1])
        except:
            print('error with {},{}'.format(i))

In [36]:
for b in range(len(lchaps)):
    print('b: ',b, ' -- ', len(lchaps[b]),len(echaps[b]))
    #for c in range(len(lchaps[b])):

b:  0  --  11 11
b:  1  --  13 13
b:  2  --  12 12
b:  3  --  14 14
b:  4  --  10 10
b:  5  --  13 13
b:  6  --  11 11
b:  7  --  16 16
b:  8  --  16 16
b:  9  --  6 6
b:  10  --  8 8
b:  11  --  10 10
b:  12  --  10 10
b:  13  --  12 12
b:  14  --  6 6
b:  15  --  6 6
b:  16  --  10 10
b:  17  --  16 16
b:  18  --  6 6
b:  19  --  10 10


In [38]:
alignments = []
for b in range(len(lchaps)):
    print('b: ',b)
    for c in range(len(lsects[b])):
        print('c: ',c)
        lch = lchaps[b][c]
        lsec = lsects[b][c]
        ech = echaps[b][c]
        esec = esects[b][c]
        
        try:
            ls = get_sentences(lch,lsec)
            es = get_sentences(ech,esec)
        except:
            pass
        ls = [re.sub(r'[0-9]{1,2}\.','',l) for l in ls]
        es = [re.sub(r'[0-9]{1,2}\.','',e) for e in es]
        ls = [re.sub(r'\. \. \.','',l) for l in ls]
        es = [re.sub(r'\. \. \.','',e) for e in es]
        ls =  [re.sub('[:;,]','',l) for l in ls]
        es = [ re.sub('[:;,]','',e) for e in es]

        epar = [sent_tokenize(e.replace('\n',' ')) for e in es]
        lpar = [s_tokenizer.tokenize_sentences(l.replace('\n',' ')) for l in ls]
        if len(lpar) != len(epar):
            print(len(lpar),len(epar))
            continue
        sec_alignments = []
        for p in range(len(lpar)):
            sec_alignments.append(get_sec_alignments(lpar[p],epar[p]))
        alignments.append(sec_alignments)

b:  0
c:  0
10 11
c:  1
8 11
c:  2
c:  3
c:  4
c:  5
c:  6
c:  7
c:  8
c:  9
c:  10
b:  1
b:  2
c:  0
c:  1
c:  2
c:  3
c:  4
c:  5
c:  6
c:  7
c:  8
c:  9
c:  10
c:  11
b:  3
c:  0
c:  1
c:  2
c:  3
error, []
error, []
error, []
error, []
c:  4
c:  5
c:  6
c:  7
c:  8
c:  9
c:  10
c:  11
c:  12
c:  13
b:  4
c:  0
c:  1
c:  2
c:  3
c:  4
c:  5
c:  6
c:  7
c:  8
c:  9
b:  5
c:  0
c:  1
c:  2
c:  3
c:  4
3 2
c:  5
c:  6
c:  7
c:  8
c:  9
c:  10
c:  11
c:  12
b:  6
c:  0
c:  1
c:  2
c:  3
c:  4
c:  5
c:  6
c:  7
c:  8
c:  9
c:  10
b:  7
c:  0
c:  1
c:  2
c:  3
c:  4
c:  5
c:  6
c:  7
c:  8
1 9
c:  9
c:  10
c:  11
c:  12
c:  13
c:  14
c:  15
b:  8
c:  0
c:  1
c:  2
c:  3
c:  4
c:  5
c:  6
c:  7
c:  8
c:  9
3 2
c:  10
c:  11
c:  12
c:  13
c:  14
c:  15
b:  9
c:  0
c:  1
c:  2
c:  3
c:  4
c:  5
b:  10
c:  0
c:  1
c:  2
c:  3
15 14
c:  4
c:  5
9 6
c:  6
c:  7
b:  11
c:  0
c:  1
c:  2
c:  3
c:  4
c:  5
c:  6
c:  7
16 19
c:  8
c:  9
b:  12
c:  0
c:  1
3 4
c:  2
c:  3
c:  4
c:  5
c:  6
c:  7
c: 

In [39]:
count = 0
for a in alignments:
    for s in a:
        for p in s:
            count += len(p)

In [40]:
count

6350

In [44]:
alignments[-1]

[[(['Igitur Symmachus scolasticus Iudaeus iubente non rege sed tyranno dictavit praecepta die quarta feria septimo kalend.'],
   'Then Symmachus an advocate and a Jew at the order of a tyrant rather than a king announced on an appointed day which was a Wednesday the 26th of August in the fourth indiction under the consulship of Olybrius that on the following Sabbath the Arians would take possession of the Catholic churches.'),
  (['Septembr.'],
   'But He who does not allow his faithful worshippers to be oppressed by unbelievers soon brought upon Theodoric the same punishment that Arius the founder of his religion had suffered for the king was seized with a diarrhea and after three days of open bowels lost both his throne and his life on the very same day on which he rejoiced to attack the churches.'),
  (['indictione quarta Olybrio consule ut die dominico adveniente Arriani basilicas catholicas invaderent.'],
   'But before breathing his last he named his grandson Athalaric as his suc

In [78]:
import json

In [46]:
f = open('aligned_sentences/amm_lat_sentences.json','w') #amm_lat.xml
j = json.dumps(alignments)
f.write(j)
f.close()
                

In [945]:
es = get_sentences(echaps[0][6],esects[0][6])
ls = get_sentences(lchaps[0][6],lsects[0][6])

In [19]:
lsects[0][0]

[<milestone n="1" unit="section"></milestone>,
 <milestone n="2" unit="section"></milestone>,
 <milestone n="3" unit="section"></milestone>,
 <milestone n="4" unit="section"></milestone>,
 <milestone n="5" unit="section"></milestone>,
 <milestone n="6" unit="section"></milestone>,
 <milestone n="7" unit="section"></milestone>,
 <milestone n="8" unit="section"></milestone>,
 <milestone n="9" unit="section"></milestone>,
 <milestone n="10" unit="section"></milestone>,
 <milestone n="1" unit="section"></milestone>,
 <milestone n="2" unit="section"></milestone>,
 <milestone n="3" unit="section"></milestone>,
 <milestone n="4" unit="section"></milestone>,
 <milestone n="5" unit="section"></milestone>,
 <milestone n="6" unit="section"></milestone>,
 <milestone n="7" unit="section"></milestone>,
 <milestone n="8" unit="section"></milestone>,
 <milestone n="9" unit="section"></milestone>,
 <milestone n="10" unit="section"></milestone>,
 <milestone n="11" unit="section"></milestone>,
 <mileston

In [847]:
esects[0][6][0]['n']

'1'

In [798]:
list(echaps[0][6].children)[3].find('milestone')

<milestone n="1" unit="section"></milestone>

In [ ]:
for i in range(len(lp)):
    print(lp[i]['n'],ep[i]['n'])

In [ ]:
lp

In [ ]:
for i in range(len(lb)):
    lp = lb[i].findAll('div2', {'type':'chapter'})
    ep = eb[i].findAll('div2', {'type':'chapter'})
    print(len(lp),len(ep))

In [199]:
s = lchaps[2][1].findAll('milestone')[0]
te = []
t = []
i = 1
while s.next_sibling != None:
    s = s.next_sibling
    if s.name == 'milestone':
        print(s['n'])
        if int(s['n']) == int(lsects[2][1][i][0]['n']):
            i += 1
            te.append(' '.join(t))
            t = []
    if s.name in [None,'corr', 'name']:
        try:
            t.append(s.text.strip())
        except:
            t.append(s.strip())
    #  lchaps[2][1].findAll('milestone')[-1].next_sibling.next_sibling

2
3
4
5
6
7
8
9
10
11
12
13
